In [46]:
from unittest import result
from attr import attr
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import math
import os


In [2]:
item_name = []
prices = []

In [86]:
# open new soup with a new url
def get_page(url):
    response =  requests.get(url)
    if not response.ok:
        print('server responded: ', response.status_code)
    else:
        data=response.text
        soup=BeautifulSoup(data,"html.parser")
        return soup

# use in the individual product page to find details
def get_detail_data(soup):
    try:
        title = soup.find('h1', attrs = {'class':"x-item-title__mainTitle"}).find('span').text
    except:
        title =' '

    try:
        price = soup.find('div', attrs = {'class':"mainPrice"}).find('span')
        price = price.get('content')
    except:
        price =' '

    try: 
        sold = soup.find('div', attrs = {'id':"why2buy"}).find('span').text #.split(' ')[0]
    except:
        sold =' '

    try: 
        review = soup.find('div', attrs = {'class':"overlay-top"}).find('a') #.split(' ')[0]
        reviewURL = review.get("href") + '&pgn='
        review_num = int(re.search(r'\d+', review.text).group())
        page = int(math.ceil(review_num/10))
        print(reviewURL,page)
    
        reviews = get_review(reviewURL, page)
        
    except:
        reviews =' '

    try:
        descs = soup.find('div', attrs ={'data-testid':"ux-layout-section__item",'class':"ux-layout-section__item ux-layout-section__item--table-view"}).find_all('span', attrs={'class':"ux-textspans"})
    except:
        descs = [' ']
    finally:
        specs = []
        for i in range(len(descs)):
            descs = [i for i in descs if i.contents[1] !='Read more'] #remove read more expandable tag
        for i in range(len(descs)-1):
            if i % 2 ==0: 
                specs.append([descs[i].contents[1],descs[i+1].contents[1]])
        if specs[0][1][:10] == specs[1][0][:10]:
            specs[0][1] = specs[1][0]
            del specs[1]
        
        
        specs_dict = {}
        for i in specs:
            specs_dict[i[0][:-1]] = i[1]
        print(specs_dict)

   
    return [title, price,sold,reviews] , specs_dict


# to see the specs of an individual product
def get_single_df(specs):
    df = pd.DataFrame(specs, columns = ['Specs title','Specs value'])
    return df

def get_review(url,page):
    print(url,page)
    reviews = []
    # url = url.split('urw')
    # url1 = 'urw/Samsung-TU7000-43-4K-LED-Smart-TV-Titan-Gray'.join(url)
    # print(url1)
    print("in get_review")
    for i in range(1):
        print(f'in loop{i}')
        
        url = url + str(0+i)
        soup = get_page(url)
        

        try: 
            review= soup.find_all('p',attrs= {'class':"review-item-content rvw-wrap-spaces", 'itemprop':"reviewBody"})
        except:
            review = ''
        finally:
            if review !="":
                reviews.extend(review)
    
    for i in range(len(reviews)):
        reviews[i] = str(reviews[i]).replace('<p class="review-item-content rvw-wrap-spaces" itemprop="reviewBody">','')
        reviews[i] = str(reviews[i]).replace('<span class="show-full-review">','')
        reviews[i] = str(reviews[i]).replace('</span>','')
        reviews[i] = str(reviews[i]).replace('</br>','')
        reviews[i] = str(reviews[i]).replace('<br/>','')
        reviews[i] = str(reviews[i]).replace('</p>','')
        reviews[i] = str(reviews[i]).replace('<a class="show-full-review-link" href="javascript:;">Read full review...</a>','')

    return reviews
        

    

    





In [96]:
search_term = []
# pass in search url of the search product page
search_term.append('oven')
ebayUrl = "https://www.ebay.com/sch/i.html?_from=R40&_nkw=smart+TV&_sacat=0&LH_All=1&rt=nc&LH_ItemCondition=1000&_pgn="+str(1)
r= requests.get(ebayUrl)
data=r.text
soup=BeautifulSoup(data,"html.parser")

# get all listings 
listings = soup.find_all('li', attrs={'class': 's-item'})
links = soup.find_all('a', class_ ='s-item__link')
items = [item.get('href') for item in links] # store the link to each product 

array = []
dict_keys = []
dict_specs = []
for i in range(len(items)-1):
    soup =  get_page(items[i+1])   #open each product's url
    results, specs_dict = get_detail_data(soup) 
    dict_specs.append(specs_dict)
    for k in specs_dict.keys():
        if k not in dict_keys: # get all possible specs found
            dict_keys.append(k)
    # print(results)
    array.append(results)  #store all the details of a product

# print(dict_keys)






# return results = ['Object's title', 'Price', 'Sold', [['Specs title', 'Specs value']]]
# return array  = [--list of results--]

{'Condition': "New: A brand-new, unused, unopened, undamaged item in its original packaging (where packaging is applicable). Packaging should be the same as what is found in a retail store, unless the item is handmade or was packaged by the manufacturer in non-retail packaging, such as an unprinted box or plastic bag. See the seller's listing for full details. ", 'Display Technology': 'LED', 'Model': 'NS-24DF310NA21', 'Maximum Resolution': 'HD (720p)', 'Refresh Rate': '60Hz', 'Features': 'Works with Amazon Alexa', 'Color': 'black', 'Smart TV Features': 'Fire TV', 'Audio/Video Inputs': 'RCA, HDMI', 'Year Manufactured': '2021', 'MPN': 'NA', 'Brand': 'Insignia', 'Manufacturer Warranty': 'n/a', 'Screen Size': '24"', 'UPC': '600603273322'}
{'Condition': "New: A brand-new, unused, unopened, undamaged item in its original packaging (where packaging is applicable). Packaging should be the same as what is found in a retail store, unless the item is handmade or was packaged by the manufacturer i

In [98]:

df_test = pd.DataFrame([[None]*len(dict_keys)]*len(dict_specs),columns=dict_keys)
i = 0
for dict in dict_specs:
    for k,v in dict.items():
        df_test.loc[i][k]= v
    i += 1
print(df_test)

                                            Condition Display Technology  \
0   New: A brand-new, unused, unopened, undamaged ...                LED   
1   New: A brand-new, unused, unopened, undamaged ...                LED   
2   New: A brand-new, unused, unopened, undamaged ...                LED   
3   New: A brand-new, unused, unopened, undamaged ...                LED   
4   New: A brand-new, unused, unopened, undamaged ...                LED   
..                                                ...                ...   
67  New: A brand-new, unused, unopened, undamaged ...                LED   
68  New: A brand-new, unused, unopened, undamaged ...                LED   
69  New: A brand-new, unused, unopened, undamaged ...                LED   
70  New: A brand-new, unused, unopened, undamaged ...                LED   
71  New: A brand-new, unused, unopened, undamaged ...               QLED   

                 Model Maximum Resolution Refresh Rate  \
0       NS-24DF310NA21       

In [99]:
df_review = pd.DataFrame(array, columns = ['Title','Price','Sold','Reviews'])

print(df_review)

                                                Title   Price  \
0   Insignia 24" HD 720p Class F20 Series LED Smar...  108.98   
1   TCL 32" Class HD LED Android Smart TV 3-Series...  159.99   
2   NEW ROKU Smart TV 40" FHD LED H4030F Series 40...           
3   Samsung - 43" Class 7 Series LED 4K UHD Smart ...   275.0   
4   Onn 75" Class 4k UHD (2160p) LED Frameless Rok...   599.0   
..                                                ...     ...   
67  Toshiba 32 inch 720p HD Smart LED TV Fire TV E...  269.99   
68  Samsung 32" Class Q50R QLED Smart 4K UHD TV wi...   799.0   
69  TCL 43" 4-SERIES 4K UHD HDR ROKU SMART TV 43S4...           
70  Samsung UN50AU8000 50" Ultra HD LED Smart TV -...   635.0   
71  Samsung QN32Q50R 32" QLED 4K Smart TV - Charco...   625.0   

                   Sold                                            Reviews  
0              143 sold                                                     
1               28 sold                                          

In [100]:
df = pd.concat([df_review, df_test], axis=1)
print(df)


                                                Title   Price  \
0   Insignia 24" HD 720p Class F20 Series LED Smar...  108.98   
1   TCL 32" Class HD LED Android Smart TV 3-Series...  159.99   
2   NEW ROKU Smart TV 40" FHD LED H4030F Series 40...           
3   Samsung - 43" Class 7 Series LED 4K UHD Smart ...   275.0   
4   Onn 75" Class 4k UHD (2160p) LED Frameless Rok...   599.0   
..                                                ...     ...   
67  Toshiba 32 inch 720p HD Smart LED TV Fire TV E...  269.99   
68  Samsung 32" Class Q50R QLED Smart 4K UHD TV wi...   799.0   
69  TCL 43" 4-SERIES 4K UHD HDR ROKU SMART TV 43S4...           
70  Samsung UN50AU8000 50" Ultra HD LED Smart TV -...   635.0   
71  Samsung QN32Q50R 32" QLED 4K Smart TV - Charco...   625.0   

                   Sold                                            Reviews  \
0              143 sold                                                      
1               28 sold                                        

In [101]:
#export into a json file for cleaning

if not os.path.exists("./outputs/ebay"):
    os.makedirs("./outputs/ebay")

df.to_csv(f"./outputs/ebay/TV.csv", index=False)